In [ ]:
# insert data into hbase
import csv
import happybase
import time

batch_size = 1000
host = "hbase"
#namespace = "sample_data"
#row_count = 0
#start_time = time.time()
table_name = b"rfic"

def connect_to_hbase():
    """ Connect to HBase server.
    This will use the host, namespace, table name, and batch size as defined in
    the global variables above.
    """
    _conn = happybase.Connection(host = host)
    _conn.open()
    tables = _conn.tables()
    print( "HBase has tables {0}".format(tables))

    #if table_name not in tables:
    #  print( "Creating table {0}".format(table_name))
    #  _conn.create_table(table_name, { 'data': dict() } )


    table = _conn.table(table_name)
    batch = table.batch(batch_size = batch_size)
    return _conn, batch


conn, batch = connect_to_hbase()
print("table_name = ", table_name)
table = conn.table(table_name)
rows = table.scan(row_start='2017/01/01_00:00', row_stop='2018/04/30_23:59')
outfile = open("out.csv","w")
outfile.write("Key, Date, TimeStamp, OpenValue, HighValue, LowValue, CloseValue, Volume\n")  # Writing header


n=0
while(True):
  try: 
    n = n+1
        
    r=next(rows)
    dk = r[0].decode("latin-1").replace("_", " ")
    dt = r[0].decode("latin-1").replace("_", ",")
    s=[r[1][b'data:open'], r[1][b'data:high'] ,r[1][b'data:low'], r[1][b'data:close'], r[1][b'data:volume']]
    vstring = dk + ',' + dt + ',' + ','.join([i.decode("latin-1") for i in s])
    outfile.write("%s\n" % vstring)
  except:
    break

print("Total count: ", n)
outfile.close()
conn.close()